In [7]:
import os
import shutil

while not os.path.isfile("README.md"):
    %cd ..

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import compute_sample_weight
import lightgbm as lgb
from lib.preprocess import Preprocess

pd.set_option('display.max_colwidth', None)
BASE_PATH = os.path.abspath("")

'/home/nakagawa/fdua_finance_competition'

In [15]:
%pip install plyer
from plyer import notification

notification.notify(
    title="From Python",
    message="Executed Successfully",
    app_name='Python',
    app_icon=os.path.join(BASE_PATH,'lib/notification.ico'),
)

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
